In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import os

### What we need to analyze
* User Name
* Review Text
* Review Star
* Date


In [3]:
with open("C:/Users/Admin/Desktop/Python_DA/Crawling/1.html", "r", encoding = "utf-8") as reader:
    html_string = reader.read()
    soup = BeautifulSoup(html_string, "html.parser")

In [4]:
# Review Star
soup.find_all(class_="review-rating")[0].text.strip()

'5.0 out of 5 stars'

In [5]:
# Review Text
soup.find_all(class_="a-row a-spacing-small review-data")[0].text.strip()

"At first when I got it, I assumed everything was fine, but i quickly noticed that there were a couple things off about the controller. It might have been my paranoia, but I dont think these controllers are authentic. My right analog stick has stopped working within 4 months of me buying the controller. I had my original controller for years before it stopped working, and this one didn't even last half a year. I don't understand how that is possible. I bought one from Best Buy on Black Friday and compared, and there are minute details that are off with the touch pad and the directional pad. Additionally, the serial number label on the back of the controller is different than the other controllers I have (two I bought from Best Buy a couple days ago). I recommend buying from the store...these are not the wave!"

### We will use
*soup.find_all(class_="celwidget") so we can crawl out each block comment, and then we will collect information inside each block through "FOR" loop below

In [82]:
files = os.listdir("C:/Users/Admin/Desktop/Python_DA/Crawling/")

In [83]:
review_star = []
review_text = []
review_date = []

for i in files:
    #print(i) for checking
    with open(f"C:/Users/Admin/Desktop/Python_DA/Crawling/{i}", "r", encoding = "utf8") as reader:
        html_string = reader.read()
        soup = BeautifulSoup(html_string, "html.parser")
        for review in soup.find_all(class_="a-section celwidget"):
            review_star.append(review.find_all(class_="review-rating")[0].text.strip())
            review_text.append(review.find_all(class_="review-text-content")[0].text.strip())
            review_date.append(review.find_all(class_="a-size-base a-color-secondary review-date")[0].text.strip()) #Date of Review


In [84]:
#checking
print(len(review_star))
print(len(review_text))

1990
1990


In [85]:
df = pd.DataFrame(data={"Star":review_star,"Text":review_text,"Date":review_date})
df.head()

,Star,Text,Date
0,1.0 out of 5 stars,"At first when I got it, I assumed everything w...","Reviewed in the United States on November 25, ..."
1,1.0 out of 5 stars,For some reason these controllers from Amazon ...,"Reviewed in the United States on July 17, 2018"
2,1.0 out of 5 stars,Bought this as a backup for my original PS4 co...,"Reviewed in the United States on September 22,..."
3,1.0 out of 5 stars,The left analog stick always wears out. You ma...,"Reviewed in the United States on September 17,..."
4,2.0 out of 5 stars,I have gone through two Dualshock 4 new model ...,"Reviewed in the United States on May 29, 2018"


In [86]:
df["Date"] = df["Date"].str.replace("Reviewed in the United States on ","",regex = True)

In [164]:
df.to_excel("Sentiment_analysis.xlsx", index = False)

In [204]:
df = pd.read_excel("Sentiment_analysis.xlsx")
df.head()

,Star,Text,Date
0,1.0 out of 5 stars,At first when I got it I assumed everything wa...,"November 25, 2018"
1,1.0 out of 5 stars,For some reason these controllers from Amazon ...,"July 17, 2018"
2,1.0 out of 5 stars,Bought this as a backup for my original PS con...,"September 22, 2018"
3,1.0 out of 5 stars,The left analog stick always wears out You may...,"September 17, 2018"
4,2.0 out of 5 stars,I have gone through two Dualshock new model c...,"May 29, 2018"


I'm not learn AI and Machine Learning yet, so we will using a model from Hugging Face, is not 100% accurate but i think it will work in this case
* It's still have some issues that i have no idea how to fix in the model, so i decided to removed some data out of the data set

In [20]:
import tensorflow as tf

In [21]:
from transformers import pipeline

In [ ]:
classifier = pipeline("sentiment-analysis")

In [26]:
#Example
print(classifier('This is a bad product.'))
print(classifier('Oh man, i love this thing.'))

[{'label': 'NEGATIVE', 'score': 0.999817967414856}]
[{'label': 'POSITIVE', 'score': 0.9998740553855896}]


In [210]:
df["Text"] = df["Text"].str.replace("[^a-zA-Z\s]+|https","",regex= True)
df["Text"] = df["Text"].str.replace("^(\s+)","",regex= True)

In [211]:
df.head()

,Star,Text,Date
0,1.0 out of 5 stars,At first when I got it I assumed everything wa...,"November 25, 2018"
1,1.0 out of 5 stars,For some reason these controllers from Amazon ...,"July 17, 2018"
2,1.0 out of 5 stars,Bought this as a backup for my original PS con...,"September 22, 2018"
3,1.0 out of 5 stars,The left analog stick always wears out You may...,"September 17, 2018"
4,2.0 out of 5 stars,I have gone through two Dualshock new model c...,"May 29, 2018"


In [217]:
sentiment_label = []
sentiment_score = []
#counter = 0
for review in df["Text"]:
    classification = classifier(review, max_length = 2048)
    sentiment_label.append(classification[0].get("label"))
    sentiment_score.append(classification[0].get("score"))
    #counter +=1
    #print(counter)


#### Checking and remove some data

In [ ]:
#classifier(df["Text"][1561])

In [214]:
#df.drop([1561], axis = 0, inplace = True)

In [219]:
sentiment_data = pd.DataFrame(data={"Sentiment label":sentiment_label,"Sentiment score":sentiment_score})
sentiment_data.head()

,Sentiment label,Sentiment score
0,NEGATIVE,0.998424
1,NEGATIVE,0.999487
2,NEGATIVE,0.999283
3,NEGATIVE,0.998017
4,NEGATIVE,0.999705


In [242]:
df2 = pd.concat([df,sentiment_data], axis = 1)
df2

,Star,Text,Date,Sentiment label,Sentiment score
0,1.0 out of 5 stars,At first when I got it I assumed everything wa...,"November 25, 2018",NEGATIVE,0.998424
1,1.0 out of 5 stars,For some reason these controllers from Amazon ...,"July 17, 2018",NEGATIVE,0.999487
2,1.0 out of 5 stars,Bought this as a backup for my original PS con...,"September 22, 2018",NEGATIVE,0.999283
3,1.0 out of 5 stars,The left analog stick always wears out You may...,"September 17, 2018",NEGATIVE,0.998017
4,2.0 out of 5 stars,I have gone through two Dualshock new model c...,"May 29, 2018",NEGATIVE,0.999705
...,...,...,...,...,...
1978,5.0 out of 5 stars,Bought this as a gift Felt sturdy Was a cool r...,"July 29, 2020",POSITIVE,0.999348
1979,5.0 out of 5 stars,The controller is lightweight and fits nicely ...,"December 21, 2018",NEGATIVE,0.999293
1980,3.0 out of 5 stars,I like it and it was what I needed but after a...,"July 6, 2020",NEGATIVE,0.999555
1981,2.0 out of 5 stars,It wouldnt hold any charge when plugged in to ...,"May 15, 2020",POSITIVE,0.993690


In [245]:
df2.loc[df2["Sentiment label"].isnull()]

,Star,Text,Date,Sentiment label,Sentiment score
1982,5.0 out of 5 stars,Bought as a gift for a friendI have not used i...,"August 8, 2017",NaN,NaN


In [250]:
df2.drop([1982])

,Star,Text,Date,Sentiment label,Sentiment score
0,1.0 out of 5 stars,At first when I got it I assumed everything wa...,"November 25, 2018",NEGATIVE,0.998424
1,1.0 out of 5 stars,For some reason these controllers from Amazon ...,"July 17, 2018",NEGATIVE,0.999487
2,1.0 out of 5 stars,Bought this as a backup for my original PS con...,"September 22, 2018",NEGATIVE,0.999283
3,1.0 out of 5 stars,The left analog stick always wears out You may...,"September 17, 2018",NEGATIVE,0.998017
4,2.0 out of 5 stars,I have gone through two Dualshock new model c...,"May 29, 2018",NEGATIVE,0.999705
...,...,...,...,...,...
1977,2.0 out of 5 stars,Right analog stick is already starting to drif...,"July 11, 2019",POSITIVE,0.999404
1978,5.0 out of 5 stars,Bought this as a gift Felt sturdy Was a cool r...,"July 29, 2020",POSITIVE,0.999348
1979,5.0 out of 5 stars,The controller is lightweight and fits nicely ...,"December 21, 2018",NEGATIVE,0.999293
1980,3.0 out of 5 stars,I like it and it was what I needed but after a...,"July 6, 2020",NEGATIVE,0.999555


In [251]:
df2.to_excel("Amazon_sentiment_analysis.xlsx", index = False)